# COGS 118B - Final Project

# Automated Shift Scheduling

## Group members

- Immanuel Chai
- Sidney Ma

# Abstract 
The goal of the project is to automate and optimally create a scheduler for a dining hall with 100 student workers each with a randomly generated class schedule while improving the schedule over time through deep learning. The data used is a csv file that contains a list of 100 students alongside their availability for every half hour in the day starting from 6:00 am to 11:30 pm. Initially, the solution involves creating a scheduler that assigns students to shifts based on the basic requirements. The scheduler eventually optimizes to meet the other constraints. The final schedule is saved to an excel file where it highlights when each student has a shift and the scheduler is measured based on how well it can stick to the constraints while also keeping the basic requirements.

# Background

The workforce scheduling problem (WSP) involves determining the optimal assignment of workers to various work periods while satisfying constraints such as labor laws and employee preferences<sup>([1](https://www.researchgate.net/publication/2333359_Workforce_Scheduling))</sup>. This problem is crucial in maximizing organizational efficiency and profitability without compromising staff well-being.

WSP has been a significant theme in operations research due to its broad applications in industries reliant on human capital, such as healthcare, public transportation, and customer service<sup>([2](https://www.researchgate.net/publication/46189793_Heuristic_Algorithm_for_Workforce_Scheduling_Problems))</sup>. These industries require efficient scheduling to ensure high service levels and operational efficiency. In the context of higher education, one application of the WSP can be found in assigning shifts for student workers at the dining hall. This setting presents unique challenges, such as students’ class schedules, ensuring adequate staffing in opening and closing shift hours, and preventing consecutive closing and opening shifts.

Previous research has applied various algorithms and methodologies to address WSP. Techniques used for Constraint-Satisfaction Problems (CSPs) are well-suited for handling the complex and diverse constraints typical of scheduling problems<sup>([3](http://eil2.mie.utoronto.ca/wp-content/uploads/scheduling/papers/fox-ecai90.pdf))</sup>. Machine learning methods, particularly reinforcement learning, have also shown promise in dynamically adjusting schedules based on real-time data and learning from historical scheduling outcomes<sup>([4](https://www.sciencedirect.com/science/article/pii/S2212827122001846))</sup>.

# Problem Statement

This problem is an approximation of the needs of managers at the UCSD Sixth Market, where they have to assign shifts for student workers each week, based on their class schedules and a set of other constraints. Our goal is to create a tool that takes in a large dataset of student workers’ class schedules and outputs a realistic, fair, and efficient work shift schedule for a given facility. Specifically, the tool will be provided with 100 unique students and their weekly class schedules. It will need to organize a schedule such that at least 6 students are present in the facility at all open times (6am - 12 am), each shift is 4.5 hours long, each student works 3 or 4 shifts per week, and does not work more than 1 shift per day. 

As mentioned before, this problem could be appropriately modeled as a CSP. The tool’s performance will be based on how well it abides by the above constraints, measured by its reward scores. We will determine the exact preferences in the process. We also plan to replicate its performance with a wide variety of student schedules to ensure that it is generalizable to new students.

# Data

In order to train and test our tool, we will need student class schedule data to work with. While it could be possible to collect real student schedule data, this data might not be readily available to us, and it might lead to a considerable privacy violation. To avoid this, we plan to create a synthetic dataset which will model what typical UCSD student class schedules look like. Each observation of this dataset represents a student and their weekly class schedule. This includes their ID, name, and hour-by-hour availability from Sunday through Saturday, 6:00am to 11:30pm. A rough example model of this dataset can be found here: https://github.com/iachai/COGS188_schedulers/blob/main/example.csv.

While it could also be possible to collect data on real shift schedules to get a sense of what a “good schedule” looks like, this involves the same problem as before, with access and privacy issues. Most likely, we will not need this data anyway, as our solution does not involve modeling after real schedules.


In [ ]:
import pandas as pandas
file_path = 'example.csv'
students_df = pd.read_csv(file_path)
print(students_df.head())

# Proposed Solution

We will test out two types of algorithms for this problem. 
1. **Constraint Satisfaction Problem (CSP)**:
* We will take each students’ assigned shifts as a variable, and the possible shift slots as the domain. The basic constraints are that (1) each student can only work 3 or 4 shifts per week, (2) between 6am - 12am, at least 6 student workers will always be present, (3) no shifts can overlap with class times, and (4) a shift always starts on the clock or at half and hour. 
* We first assign shifts to the student with least available time slots, and recursively assign shifts to the next student. We will propagate this updated constraint to the domains of remaining students. If there are any conflicts, we will backtrack to the last assignment. Finally, we check that all students are assigned shifts without conflicts, and verify our solution. 
2. **Reinforcement Learning with Q-learning**:
* We will define the states as the current schedule configuration, and actions as assigning a shift. We will give rewards for satisfying each constraint (e.g., add score for having 3 or 4 shifts per week, minus score for having consecutive shifts). We will test out different learning rates and epsilon scores to find the optimal policy. Finally, we check that all students are assigned shifts without conflicts. 

# Evaluation Metrics

For each schedule result, we will test the following metrics on the training and testing dataset. 
1. **Schedule feasibility:**
* Measures the percentage of constraints satisfied by the algorithm over the total number of constraints which would include students having 3-4 working shifts, no more than one shift per day, avoid consecutive opening and closing shifts, having enough student workers during peak hours, and having 1-2 leads per shift.

* Schedule feasibility = (Number of satisfied constraints/Total constraints) x number of students
Ex: IF there are 350 satisfied constraints out of 400 total constraints, the constraint satisfaction would be 87.5%

2. **Student workload balance:** 
* How even the workload is distributed to students based on their number of shifts and their schedule. For example, the score would be low if certain students are only assigned 2 shifts and others are assigned 5. 

* Workload balance = standard deviation of number of shifts = std (1/n * sum (Si - S_mean)^2), where we have n students in total, i goes from 1 to n, and S represents the number of shifts per student per week. 
3. **Computational efficiency:** 
* Time taken to generate schedules. We want the tool to be generalized to larger facilities, so a faster calculation would have a better performance. 
4. **Lead presence score:** 
* Metric that measures the amount of shifts that have 1-2 leads present
* Lead Presence score = (Number of shifts 1-2 leads/Total number of shifts) x 100
    * Ex: If there are 100 shifts and 85 of them have 1-2 leads present, then the lead presence score would be 85%

# Results

You may have done tons of work on this. Not all of it belongs here. 

Reports should have a __narrative__. Once you've looked through all your results over the quarter, decide on one main point and 2-4 secondary points you want us to understand. Include the detailed code and analysis results of those points only; you should spend more time/code/plots on your main point than the others.

If you went down any blind alleys that you later decided to not pursue, please don't abuse the TAs time by throwing in 81 lines of code and 4 plots related to something you actually abandoned.  Consider deleting things that are not important to your narrative.  If its slightly relevant to the narrative or you just want us to know you tried something, you could keep it in by summarizing the result in this report in a sentence or two, moving the actual analysis to another file in your repo, and providing us a link to that file.

### Subsection 1

You will likely have different subsections as you go through your report. For instance you might start with an analysis of the dataset/problem and from there you might be able to draw out the kinds of algorithms that are / aren't appropriate to tackle the solution.  Or something else completely if this isn't the way your project works.

### Subsection 2

Another likely section is if you are doing any feature selection through cross-validation or hand-design/validation of features/transformations of the data

### Subsection 3

Probably you need to describe the base model and demonstrate its performance.  Maybe you include a learning curve to show whether you have enough data to do train/validate/test split or have to go to k-folds or LOOCV or ???

### Subsection 4

Perhaps some exploration of the model selection (hyper-parameters) or algorithm selection task. Validation curves, plots showing the variability of perfromance across folds of the cross-validation, etc. If you're doing one, the outcome of the null hypothesis test or parsimony principle check to show how you are selecting the best model.

### Subsection 5 

Maybe you do model selection again, but using a different kind of metric than before?



# Discussion

### Interpreting the result

OK, you've given us quite a bit of tech informaiton above, now its time to tell us what to pay attention to in all that.  Think clearly about your results, decide on one main point and 2-4 secondary points you want us to understand. Highlight HOW your results support those points.  You probably want 2-5 sentences per point.

### Limitations

Are there any problems with the work?  For instance would more data change the nature of the problem? Would it be good to explore more hyperparams than you had time for?   

### Ethics & Privacy

As mentioned before, we are creating synthetic data to train/test our tool. As such, we are not using real student data and are not concerned about privacy violations. We will also not provide any real information about students, classes, or facilities in our data. 


We do not intend to release our tool for actual use, but we will be very clear on any faults it has in our final report. We will also make it clear that it has been trained on synthetic data and not real student data, so while it will hopefully generalize to the real world, that is not what it will have been trained on.

### Conclusion

Reiterate your main point and in just a few sentences tell us how your results support it. Mention how this work would fit in the background/context of other work in this field if you can. Suggest directions for future work if you want to.

# Footnotes
<a name="footnote1">[1]:</a> [Workforce Scheduling](https://www.researchgate.net/publication/2333359_Workforce_Scheduling)<br>
<a name="footnote2">[2]:</a> [Heuristic Algorithm for Workforce Scheduling Problems](https://www.researchgate.net/publication/46189793_Heuristic_Algorithm_for_Workforce_Scheduling_Problems)<br>
<a name="footnote3">[3]:</a> [Constraint-Satisfaction Problems in Scheduling](http://eil2.mie.utoronto.ca/wp-content/uploads/scheduling/papers/fox-ecai90.pdf)<br>
<a name="footnote4">[4]:</a> [Reinforcement Learning in Scheduling](https://www.sciencedirect.com/science/article/pii/S2212827122001846)<br>
